# 2. Serialization of Python Expressions into The Wolfram Language

Continuation of the Wolfram Language™ for Python users. This notebook contains examples of:
- how to convert Python expressions into Wolfram expressions and vice versa
- which Python libraries are supported 
- how to create custom encoders/decoders to deal with further data types

## Serializing from Python into The Wolfram Language:

Expressions written in Python synatx can be converted into conventional Wolfram Language synatx with **export**.

* By default, the target_format is wl and this will produce an InputForm string.
* A target_format of wxf can also be used, which can then be deserialized.

In [1]:
from wolframclient.language import wl
from wolframclient.serializers import export
from wolframclient.deserializers import binary_deserialize
export([1,2,3])

b'{1, 2, 3}'

In [2]:
wxf = export([1, 2, 3], target_format = "wxf")
print(wxf)
binary_deserialize(wxf)

b'8:f\x03s\x04ListC\x01C\x02C\x03'


(1, 2, 3)

Export allows you to take a hybridised Python/Wolfram Language expression and produce a single unified Wolfram Language expression:

In [3]:
export(wl.Select([1,2,3], wl.PrimeQ))

b'Select[{1, 2, 3}, PrimeQ]'

## In-built data type serialization
    
- Lists, sets and dicts are serialized into wl lists.
- Generators and iterators are also evaluated into lists.
- Numerical types are serialized into their equivalent in the Wolfram Language.
- Date, Time and DateTime are serialized into DateObject/TimeObject.

In [4]:
import datetime
now = datetime.datetime.now()
export([now.time(), now.date(), now])

b'{TimeObject[{15, 2, 25.969478}], DateObject[{2021, 4, 14}, "Day", "Gregorian", None], DateObject[{2021, 4, 14, 15, 2, 25.969478}, "Instant", "Gregorian", $TimeZone]}'

## Libraries

A number of popular Python libraries have default serialization with **wolframclient** export:
    
- **PIL** images are automatically serialized into Image.
- **NumPy** arrays of signed and unsigned integers, floats and complexes are serialized into NumericArray.
- **Pandas** series and data frames are serialized into Association, Dataset or TimeSeries.

In [5]:
from PIL import Image
logo = Image.open('/tmp/logo.png')

In [6]:
import numpy
x = numpy.array([1, 2, 3])
print(x)
export(x)

[1 2 3]


b'BinaryDeserialize[ByteArray["ODrCAgEDAQAAAAIAAAADAAAA"]]'

In [7]:
import pandas
df = pandas.DataFrame(
    {'col1': ['v12', 'v12'],
    'col2': ['v21', 'v22']},
    index=['id1', 'id2'])
export(df)

b'Dataset[<|"id1" -> <|"col1" -> "v12", "col2" -> "v21"|>, "id2" -> <|"col1" -> "v12", "col2" -> "v22"|>|>]'

In [8]:
logowxf = export(Image.open('/tmp/barchart1.png'), target_format='wxf')

In [9]:
from wolframclient.evaluation import WolframLanguageSession
with WolframLanguageSession() as wl_blocksession:
    print(wl_blocksession.evaluate(binary_deserialize(logowxf)))

Image[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255,

## Extending Serialization

Custom serialization can be achieved through writing an encoder.

https://reference.wolfram.com/language/WolframClientForPython/docpages/advanced_usages.html#extending-serialization-writing-an-encoder
